In [2]:
import torch
import re
import sys
import spacy
import unicodedata
import numpy as np
from torch.autograd import Variable

In [3]:
sys.executable

'D:\\Anaconda3\\envs\\Torch10\\python.exe'

In [4]:
torch.__version__

'1.0.1'

In [5]:
use_cuda = ['GPU:0' if torch.cuda.is_available() else 'CPU']

In [4]:
nlp = spacy.load('en_core_web_sm', parser=False)

In [5]:
# nlp.tagger.labels

In [6]:
# {w: i for i, w in enumerate([''] + list(nlp.tagger.labels))}

In [7]:
# {w: i for i, w in enumerate([''] + list(nlp.entity.move_names))}

In [8]:
sentence='Beijing like NLP.'

In [9]:
raw_tokens = nlp(sentence); raw_tokens

Beijing like NLP.

In [10]:
for token in raw_tokens:
    print(token, '||', token.is_punct, token.is_space, token.ent_type_, token.tag_)

Beijing||FalseFalseGPENNP
like||FalseFalseIN
NLP||FalseFalseORGNNP
.||TrueFalse.


In [11]:
np.random.uniform(-1, 1, size=(5, 5))

array([[ 0.79698305,  0.83702857, -0.80461536,  0.55461405,  0.48790577],
       [-0.94797244,  0.48171392, -0.46968296, -0.31926014,  0.29292209],
       [-0.91648382,  0.73219536, -0.65105157,  0.60447115,  0.62223631],
       [-0.96032186, -0.52267581,  0.12606397, -0.43089066, -0.9127302 ],
       [-0.30685928, -0.36128405,  0.10142948, -0.60657699, -0.52494669]])

In [12]:
test = [x for x in range(1, 11)]

In [13]:
test[0: -2]

[1, 2, 3, 4, 5, 6, 7, 8]

In [14]:
# -2只是索引位置

In [15]:
def char2id_sent(sent, c2id, unk_id=None, to_lower=False):
    if to_lower:
        sent = sent.lower()
    cids = [[c2id['<STA>'] + [c2id[c] if c in c2id else unk_id for c in w] + [c2id['<END>']]] for w in sent]
    return cids

In [16]:
from collections import Counter

In [17]:
context = "Don't break my heart."

In [18]:
doc = nlp(context)

Do do
n't not
break break
my -PRON-
heart heart
. .


In [19]:
lemma = {token.lemma_ if token.lemma_ != '-PRON-' else token.text.lower() for token in doc}

In [20]:
lemma

{'.', 'break', 'do', 'heart', 'my', 'not'}

In [21]:
lemma_ = {token.text.lower() for token in doc}  # 未拿回单词原型

In [22]:
lemma_

{'.', 'break', 'do', 'heart', 'my', "n't"}

In [23]:
_lemma = {token.lemma_ for token in doc}

In [24]:
_lemma

{'-PRON-', '.', 'break', 'do', 'heart', 'not'}

In [25]:
question_word = {token.text for token in doc}; question_word

{'.', 'Do', 'break', 'heart', 'my', "n't"}

In [26]:
question_lemma = {token.lemma_ if token.lemma_ != '-PRON-' else token.text for token in doc}; question_lemma

{'.', 'break', 'do', 'heart', 'my', 'not'}

In [27]:
match_origin = [token.text in question_word for token in doc]; match_origin

[True, True, True, True, True, True]

In [28]:
def feature_gen(context, question):
    counter_ = Counter(w.text.lower() for w in context)
    total = sum(counter_.values())
    term_freq = [counter_[w.text.lower()] / total for w in context]
    question_word = {w.text for w in question}  # 集合
    question_lemma = {w.lemma_ if w != '-PRON-' else w.text.lower() for w in question}  # To lemma_, the base form of the word.
    match_origin = [w.text in question_word for w in context]
    match_lower = [w.text.lower() in question_word for w in context]
    match_lemma = [(w.lemma_ if w.lemma_ != '-PRON-' else w.text.lower()) in question_lemma for w in context]
    C_features = list(zip(term_freq, match_origin, match_lower, match_lemma))
    return C_features

In [29]:
context = "Once there was a beautiful fish named Asta. Asta lived in the ocean. There were lots of other fish in the ocean where Asta lived. They played all day long. \n\nOne day, a bottle floated by over the heads of Asta and his friends. They looked up and saw the bottle. \"What is it?\" said Asta's friend Sharkie. \"It looks like a bird's belly,\" said Asta. But when they swam closer, it was not a bird's belly. It was hard and clear, and there was something inside it. \n\nThe bottle floated above them. They wanted to open it. They wanted to see what was inside. So they caught the bottle and carried it down to the bottom of the ocean. They cracked it open on a rock. When they got it open, they found what was inside. It was a note. The note was written in orange crayon on white paper. Asta could not read the note. Sharkie could not read the note. They took the note to Asta's papa. \"What does it say?\" they asked. \n\nAsta's papa read the note. He told Asta and Sharkie, \"This note is from a little girl. She wants to be your friend. If you want to be her friend, we can write a note to her. But you have to find another bottle so we can send it to her.\" And that is what they did."

In [30]:
question = "what was the name of the fish, What looked like a birds belly"

In [31]:
question = nlp(question)

In [32]:
context = nlp(context)

In [33]:
test = torch.tensor([
    [1, 2, 4, 5],
    [2, 3, 9, 0]
]); test

tensor([[1, 2, 4, 5],
        [2, 3, 9, 0]])

In [34]:
test.shape

torch.Size([2, 4])

In [35]:
test_ = test.unsqueeze(1); test_

tensor([[[1, 2, 4, 5]],

        [[2, 3, 9, 0]]])

In [36]:
test_.shape

torch.Size([2, 1, 4])

In [37]:
test_

tensor([[[1, 2, 4, 5]],

        [[2, 3, 9, 0]]])

In [38]:
# 将张量广播到新形状

In [39]:
test_.expand([2, 2, 4])

tensor([[[1, 2, 4, 5],
         [1, 2, 4, 5]],

        [[2, 3, 9, 0],
         [2, 3, 9, 0]]])

In [40]:
test_1 = torch.arange(1, 9).resize_(2, 4); test_1

tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]])

In [41]:
# 扩展1维

In [42]:
test_1.unsqueeze_(1); test_1

tensor([[[1, 2, 3, 4]],

        [[5, 6, 7, 8]]])

In [43]:
test_1.shape

torch.Size([2, 1, 4])

In [44]:
# expand

In [45]:
test_2 = test_1.expand([2, 2, 4]); test_2

tensor([[[1, 2, 3, 4],
         [1, 2, 3, 4]],

        [[5, 6, 7, 8],
         [5, 6, 7, 8]]])

In [46]:
print(test_1.shape, test_2.shape)

torch.Size([2, 1, 4])torch.Size([2, 2, 4])


In [47]:
v1 = torch.arange(1, 5); v1

tensor([1, 2, 3, 4])

In [48]:
v2 = torch.arange(1, 4); v2

tensor([1, 2, 3])

In [49]:
torch.ger(v1, v2)

tensor([[ 1,  2,  3],
        [ 2,  4,  6],
        [ 3,  6,  9],
        [ 4,  8, 12]])

In [50]:
torch.ger(v1, v2).shape  # v1的转置乘以v2

torch.Size([4, 3])

In [51]:
# 外积结果为张量，内积结果为标量

In [52]:
test_triu = torch.ger(v1, v2).triu(); test_triu

tensor([[1, 2, 3],
        [0, 4, 6],
        [0, 0, 9],
        [0, 0, 0]])

In [53]:
test_triu = test_triu.eq(0).unsqueeze(0); test_triu

tensor([[[0, 0, 0],
         [1, 0, 0],
         [1, 1, 0],
         [1, 1, 1]]], dtype=torch.uint8)

In [61]:
# indices = list(range(len(context))); indices

In [62]:
import random

In [64]:
def bertify(self, words):
    if self.bert_tokenizer is None:
        return None
    bpe = ['[CLS]']
    x_bert_offsets = []
    for word in words:
        now = self.bert_tokenizer.tokenize(word)
        x_bert_offsets.append([len(bpe), len(bpe) + len(now)])
        bpe.extend(now)
    bpe.append('[SEP]')
    x_bert = self.bert_tokenizer.convert_tokens_to_ids(bpe)
    return x_bert, x_bert_offsets

In [66]:
x = torch.LongTensor(1, 15).fill_(0); x

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [69]:
y = torch.LongTensor(1, 15).fill_(-1); y

tensor([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])

In [8]:
x = torch.LongTensor([1, 10]); x

tensor([ 1, 10])

In [10]:
x_ = torch.tensor([32, 2], dtype=torch.long); x_

tensor([32,  2])

In [24]:
import json
from tqdm import tqdm

In [25]:
with open(file='E:\PycharmProjects\SDNetMyself\coqa\data\coqa-dev-v1.0.json', mode='r', encoding='utf-8') as f:
    dataset = json.load(f)

In [26]:
print('Processing json...')
data = []
tot = len(dataset['data'])
print(tot)
for data_idx in tqdm(range(tot)):
    datum = dataset['data'][data_idx]
    print(datum)
    break

Processing json...
500
  0%|          | 0/500 [00:00<?, ?it/s]{'source': 'mctest', 'id': '3dr23u6we5exclen4th8uq9rb42tel', 'filename': 'mc160.test.41', 'story': 'Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer\'s horses slept. But Cotton wasn\'t alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton\'s mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer\'s orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing. \n\n"What are you doing, Cotton?!" \n\n"I only wanted to

In [6]:
def gen_upper_triangle(score_s, score_e, max_len, use_cuda):
    batch_size = score_s.shape[0]
    context_len = score_s.shape[1]
    # batch x context_len x context_len
    expand_score = score_s.unsqueeze(2).expand([batch_size, context_len, context_len]) + \
                   score_e.unsqueeze(1).expand([batch_size, context_len, context_len])
    print(expand_score.shape)
    score_mask = torch.ones(context_len)
    if use_cuda:
        score_mask = score_mask.cuda()
    score_mask = torch.ger(score_mask, score_mask).triu().tril(max_len - 1)  # 向量间外积
    empty_mask = score_mask.eq(0).unsqueeze(0).expand_as(expand_score)
    expand_score.data.masked_fill_(empty_mask.data, -float('inf'))
    return expand_score.contiguous().view(batch_size, -1)

In [40]:
batch_size = 4
context_len = 8

In [42]:
score_mask = torch.ones(context_len); score_mask

tensor([1., 1., 1., 1., 1., 1., 1., 1.])

In [43]:
score_mask = torch.ger(score_mask, score_mask); score_mask

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [53]:
score_mask.triu().tril(7)

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1.]])

In [54]:
empty_mask = score_mask.eq(0).unsqueeze(0).expand_as(expand_score); empty_mask

tensor([[[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0

In [57]:
expand_score

tensor([[[1.5604, 0.7544, 1.5624, 0.8054, 1.1828, 0.8934, 0.7587, 1.5940],
         [1.0262, 0.2202, 1.0282, 0.2712, 0.6486, 0.3592, 0.2246, 1.0598],
         [1.3542, 0.5482, 1.3562, 0.5992, 0.9766, 0.6872, 0.5526, 1.3878],
         [1.1037, 0.2977, 1.1057, 0.3487, 0.7261, 0.4367, 0.3021, 1.1373],
         [1.2669, 0.4610, 1.2690, 0.5119, 0.8893, 0.5999, 0.4653, 1.3005],
         [1.4550, 0.6490, 1.4570, 0.7000, 1.0774, 0.7880, 0.6533, 1.4886],
         [1.2867, 0.4807, 1.2887, 0.5317, 0.9091, 0.6197, 0.4850, 1.3203],
         [0.9663, 0.1604, 0.9684, 0.2113, 0.5888, 0.2993, 0.1647, 0.9999]],

        [[1.9078, 1.2668, 1.5963, 1.4643, 1.1351, 1.6888, 1.7552, 1.0667],
         [1.8334, 1.1923, 1.5219, 1.3898, 1.0606, 1.6143, 1.6808, 0.9922],
         [1.7618, 1.1207, 1.4503, 1.3182, 0.9890, 1.5427, 1.6092, 0.9206],
         [1.4660, 0.8249, 1.1545, 1.0224, 0.6932, 1.2469, 1.3134, 0.6248],
         [1.5663, 0.9253, 1.2548, 1.1227, 0.7936, 1.3473, 1.4137, 0.7251],
         [1.1908, 0.549

In [59]:
expand_score.data.masked_fill_(empty_mask.data, -float('inf'))

tensor([[[1.5604, 0.7544, 1.5624, 0.8054, 1.1828, 0.8934, 0.7587, 1.5940],
         [1.0262, 0.2202, 1.0282, 0.2712, 0.6486, 0.3592, 0.2246, 1.0598],
         [1.3542, 0.5482, 1.3562, 0.5992, 0.9766, 0.6872, 0.5526, 1.3878],
         [1.1037, 0.2977, 1.1057, 0.3487, 0.7261, 0.4367, 0.3021, 1.1373],
         [1.2669, 0.4610, 1.2690, 0.5119, 0.8893, 0.5999, 0.4653, 1.3005],
         [1.4550, 0.6490, 1.4570, 0.7000, 1.0774, 0.7880, 0.6533, 1.4886],
         [1.2867, 0.4807, 1.2887, 0.5317, 0.9091, 0.6197, 0.4850, 1.3203],
         [0.9663, 0.1604, 0.9684, 0.2113, 0.5888, 0.2993, 0.1647, 0.9999]],

        [[1.9078, 1.2668, 1.5963, 1.4643, 1.1351, 1.6888, 1.7552, 1.0667],
         [1.8334, 1.1923, 1.5219, 1.3898, 1.0606, 1.6143, 1.6808, 0.9922],
         [1.7618, 1.1207, 1.4503, 1.3182, 0.9890, 1.5427, 1.6092, 0.9206],
         [1.4660, 0.8249, 1.1545, 1.0224, 0.6932, 1.2469, 1.3134, 0.6248],
         [1.5663, 0.9253, 1.2548, 1.1227, 0.7936, 1.3473, 1.4137, 0.7251],
         [1.1908, 0.549

In [58]:
score_mask.triu()

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1.]])

In [50]:
score_mask.triu().tril(7)

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1.]])

RuntimeError: ger: Expected 1-D argument self, but got 2-D

In [17]:
score_s = torch.rand(size=(batch_size, context_len)); score_s

tensor([[0.7182, 0.1840, 0.5120, 0.2615, 0.4247, 0.6128, 0.4444, 0.1241],
        [0.9564, 0.8819, 0.8103, 0.5145, 0.6148, 0.2393, 0.2514, 0.8242],
        [0.2692, 0.4595, 0.1416, 0.5726, 0.5573, 0.1026, 0.7331, 0.6917],
        [0.3519, 0.0956, 0.0534, 0.5248, 0.7816, 0.0689, 0.7479, 0.8332]])

In [18]:
score_s.shape

torch.Size([4, 8])

In [19]:
score_e = torch.rand(size=(batch_size, context_len)); score_e

tensor([[0.8422, 0.0363, 0.8443, 0.0872, 0.4646, 0.1752, 0.0406, 0.8758],
        [0.9515, 0.3104, 0.6400, 0.5079, 0.1787, 0.7324, 0.7989, 0.1103],
        [0.1705, 0.6109, 0.0538, 0.8867, 0.6410, 0.3490, 0.9653, 0.4171],
        [0.3235, 0.9449, 0.3618, 0.7546, 0.0640, 0.8573, 0.9576, 0.0769]])

In [32]:
score_e.shape

torch.Size([4, 8])

In [37]:
score_s.unsqueeze(2).shape

torch.Size([4, 8, 1])

In [35]:
test_s = score_s.unsqueeze(2).expand([batch_size, context_len, context_len]); test_s.shape

torch.Size([4, 8, 8])

In [31]:
score_e.unsqueeze(1).shape

torch.Size([4, 1, 8])

In [27]:
test_e = score_e.unsqueeze(1).expand([batch_size, context_len, context_len]); test_e.shape

torch.Size([4, 8, 8])

In [28]:
expand_score = test_s + test_e; expand_score.shape

torch.Size([4, 8, 8])

In [38]:
score_s.unsqueeze(2)

tensor([[[0.7182],
         [0.1840],
         [0.5120],
         [0.2615],
         [0.4247],
         [0.6128],
         [0.4444],
         [0.1241]],

        [[0.9564],
         [0.8819],
         [0.8103],
         [0.5145],
         [0.6148],
         [0.2393],
         [0.2514],
         [0.8242]],

        [[0.2692],
         [0.4595],
         [0.1416],
         [0.5726],
         [0.5573],
         [0.1026],
         [0.7331],
         [0.6917]],

        [[0.3519],
         [0.0956],
         [0.0534],
         [0.5248],
         [0.7816],
         [0.0689],
         [0.7479],
         [0.8332]]])

In [39]:
test_s

tensor([[[0.7182, 0.7182, 0.7182, 0.7182, 0.7182, 0.7182, 0.7182, 0.7182],
         [0.1840, 0.1840, 0.1840, 0.1840, 0.1840, 0.1840, 0.1840, 0.1840],
         [0.5120, 0.5120, 0.5120, 0.5120, 0.5120, 0.5120, 0.5120, 0.5120],
         [0.2615, 0.2615, 0.2615, 0.2615, 0.2615, 0.2615, 0.2615, 0.2615],
         [0.4247, 0.4247, 0.4247, 0.4247, 0.4247, 0.4247, 0.4247, 0.4247],
         [0.6128, 0.6128, 0.6128, 0.6128, 0.6128, 0.6128, 0.6128, 0.6128],
         [0.4444, 0.4444, 0.4444, 0.4444, 0.4444, 0.4444, 0.4444, 0.4444],
         [0.1241, 0.1241, 0.1241, 0.1241, 0.1241, 0.1241, 0.1241, 0.1241]],

        [[0.9564, 0.9564, 0.9564, 0.9564, 0.9564, 0.9564, 0.9564, 0.9564],
         [0.8819, 0.8819, 0.8819, 0.8819, 0.8819, 0.8819, 0.8819, 0.8819],
         [0.8103, 0.8103, 0.8103, 0.8103, 0.8103, 0.8103, 0.8103, 0.8103],
         [0.5145, 0.5145, 0.5145, 0.5145, 0.5145, 0.5145, 0.5145, 0.5145],
         [0.6148, 0.6148, 0.6148, 0.6148, 0.6148, 0.6148, 0.6148, 0.6148],
         [0.2393, 0.239